In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## GETTING DATA INTO COLAB

In [0]:
path = "/content/drive/My Drive/submisisons/Final year/dataset/data/"
training_sentences = open(path + "train.en" , encoding = "utf-8").read().split("\n")
tandrget_sentences = open(path + "train.sen" , encoding = "utf-8").read().split("\n")

dev_complicated_sentences = open(path + "dev.en" , encoding = "utf-8").read().split("\n")
dev_simple_sentences = open(path + "dev.sen" , encoding = "utf-8").read().split("\n")


# can't print whole list because IO limits are being exceeded
# try putting to file to see output
print(training_sentences[1])
print(target_sentences[1])

print(dev_complicated_sentences[1])
print(dev_simple_sentences[1])

Gingerbread was brought to Europe in 992 by the Armenian monk Gregory of Nicopolis ( Gregory Makar ) ( Grégoire de Nicopolis ) .
Armenian monk Gregory of Nicopolis ( Gregory Makar ) ( Grégoire de Nicopolis ) brought ginger bread to Europe in 992 .
Adjacent counties are Marin ( to the south ) , Mendocino ( to the north ) , Lake ( northeast ) , Napa ( to the east ) , and Solano and Contra Costa ( to the southeast ) .
Neighbhouring countries are Marin,to the south,  Mendocino, to the north, Lake northeast, Napa to the east and Solano and Contra Costa to the southeast.


## TOKENIZING

In [0]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

complicated = spacy.load('en')
simple = spacy.load('en')

def tokenize_complicated(sentence):
    return [tok.text for tok in complicated.tokenizer(sentence)]

def tokenize_simple(sentence):
    return [tok.text for tok in simple.tokenizer(sentence)]

C_TEXT = Field(tokenize=tokenize_complicated, fix_length = 100)
C_TEXT_DEV = Field(tokenize=tokenize_complicated, fix_length = 100)

# Can specify tensor type using "d_type". Default is long
# "fix_length" : length of padding. Default is none
# "pad_first": Default is false
# "batch_first":  Whether to produce tensors with the batch dimension first. Default: False.
S_TEXT = Field(tokenize=tokenize_simple, fix_length = 100, init_token = "<sos>", eos_token = "<eos>")
S_TEXT_DEV = Field(tokenize=tokenize_simple, fix_length = 100, init_token = "<sos>", eos_token = "<eos>")

## MAKING INTO TABULAR FORMAT

In [0]:
# STILL NEED TO ADD TEST DATA AS PER TUTORIAL. DON'T KNOW WHERE ITS NEEDED

import pandas as pd
raw_data = {'Complicated' : [line for line in training_sentences], 
            'Simple': [line for line in target_sentences]}

raw_data_dev = {'Complicated' : [line for line in dev_complicated_sentences], 
            'Simple': [line for line in dev_simple_sentences]}

df_train = pd.DataFrame(raw_data, columns=["Complicated", "Simple"])
df_dev = pd.DataFrame(raw_data_dev, columns=["Complicated", "Simple"])

df_train.to_csv("df_train.csv", index=False)
df_dev.to_csv("df_dev.csv", index=False)


# remove very long sentences and sentences where translations are 
# not of roughly equal length

# DO WE NEED BELOW THING?

# df['eng_len'] = df['English'].str.count(' ')
# df['fr_len'] = df['French'].str.count(' ')
# df = df.query('fr_len < 80 & eng_len < 80')
# df = df.query('fr_len < eng_len * 1.5 & fr_len * 1.5 > eng_len')

# associate the text in the 'English' column with the EN_TEXT field, # and 'French' with FR_TEXT
data_fields = [('Complicated', C_TEXT), ('Simple', S_TEXT)]

In [0]:
# INDEXED TO 0 TO PREVENT ERROR. TAKE CARE IN CASE OF FUTURE ERRORS.
# HAVE SENT YOU HANGOUTS LINK FOR THE RELATED ERROR WITHOUT INDEXING
train, val = torchtext.data.TabularDataset.splits(path='./', train = "df_train.csv", validation = "df_dev.csv", format='csv', fields=data_fields, skip_header = True)

In [0]:
print(type(train))
print(len(train))
# print(len(train[0]))
print(type(train[0]))
print (train[0].__dict__.keys())
print(train[283421].Simple[:])

print(type(val))
print(len(val))
# print(len(train[0]))
print(type(val[0]))
print (val[0].__dict__.keys())
print(train[15000].Simple[:])

<class 'torchtext.data.dataset.TabularDataset'>
284678
<class 'torchtext.data.example.Example'>
dict_keys(['Complicated', 'Simple'])
['The', 'part', 'of', 'the', 'potato', 'that', 'people', 'eat', 'is', 'a', 'tuber', 'that', 'grows', 'under', 'the', 'ground', '.']
<class 'torchtext.data.dataset.TabularDataset'>
16001
<class 'torchtext.data.example.Example'>
dict_keys(['Complicated', 'Simple'])
['This', 'is', 'the', 'process', 'called', 'natural', 'selection', '.']


## MAKING INTO TABULAR ALTERNATE

In [0]:
# train = torchtext.data.TabularDataset(path='./df.csv', format='csv', fields=data_fields)

## BUILDING VOCAB

In [0]:
C_TEXT.build_vocab(train,val)
S_TEXT.build_vocab(train,val)

In [0]:
print(S_TEXT.vocab.stoi["hill"])
print(C_TEXT.vocab.stoi["hill"])
print(len(S_TEXT.vocab))
print(len(C_TEXT.vocab))

3631
5031
151008
184316


## MAKING BATCH SIZE

In [0]:
train_iter = BucketIterator(train, batch_size=20, \
sort_key=lambda x: len(x.Simple), shuffle=True)

val_iter = BucketIterator(val, batch_size=20, \
sort_key=lambda x: len(x.Simple), shuffle=True)

# 1 IS PADDING. HOW TO RESOLVE IT? DIDN'T UNDERSTAND TUTORIAL
batch = next(iter(train_iter))
batch_val = next(iter(val_iter))

# print(batch.Simple)
# print(batch.Complicated)
print(len(batch.Simple))
print(len(batch.Complicated))

print(len(batch_val.Simple))
print(len(batch_val.Complicated))

# DIFFERENT LENGTHS WHY?

100
100
100
100


In [0]:
print(batch)
print(batch_val)


[torchtext.data.batch.Batch of size 100]
	[.Complicated]:[torch.LongTensor of size 100x100]
	[.Simple]:[torch.LongTensor of size 100x100]

[torchtext.data.batch.Batch of size 100]
	[.Complicated]:[torch.LongTensor of size 100x100]
	[.Simple]:[torch.LongTensor of size 100x100]


## MAKING BATCH SIZE ALTERNATE

In [0]:
# train_iter = BucketIterator.splits(train, batch_size = 20, sort_key=lambda x: len(x.Simple), 
#                                    sort_within_batch=False, shuffle = True)

# batch = next(iter(train_iter))
# # print(len(batch.Simple))
# # print(len(batch.Complicated))

## TRANSFORMER

In [0]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import math
import copy
import time
from torch.nn import functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 100):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] =  math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        # print(x.shape)
        #add constant to embedding
        seq_len = x.size(1)
        # print(seq_len)
        x = x + Variable(self.pe[:,:seq_len], requires_grad=False)
        return x


input_pad = C_TEXT.vocab.stoi['<pad>']# creates mask with 0s wherever there is padding in the input
def create_masks(input_seq, target_seq):
  input_pad = C_TEXT.vocab.stoi['<pad>']# creates mask with 0s wherever there is padding in the input
  input_msk = (input_seq != input_pad).unsqueeze(1)

# create mask as before
  target_pad = S_TEXT.vocab.stoi['<pad>']
  target_msk = (target_seq != target_pad).unsqueeze(1)
  size = target_seq.size(1) # get seq_len for matrix
  
  # BELOW STATEMENT ORIGINALLY HAD NP.ONES(1,SIZE,SIZE) BUT GOT ERROR
  # CHANGED PARAMETERS AFTER SEEING THE FUNCTION CALL
  # TAKE CARE IN CASE OF FUTURE ERRORS
  nopeak_mask = np.triu(np.ones(shape = (1, size, size)),k=1).astype('uint8')
  nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
  target_msk = target_msk & nopeak_mask

  return input_msk, target_msk

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)# calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
        
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        
    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
    
# We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        
        # RANGE WAS N. CONVERTED TO N-1 TO AVOID ERROR. LOOKOUT FOR FUTURE ERRORS
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output
      # we don't perform softmax on the output as this will be handled 
# automatically by our loss function


# ----------------------HYPERPARAMS FOR MODEL------------------------------

d_model = 512
heads = 8
N = 6
src_vocab = len(C_TEXT.vocab)
trg_vocab = len(S_TEXT.vocab)



model = Transformer(src_vocab, trg_vocab, d_model, N, heads)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
        
# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation

optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


def train_model(epochs, print_every=1):
    
    # SEE POSITIONING OF BELOW 4 STATEMENTS WRT EACH EPOCH LOSS PRINTING FOR
    # TRAINGING AND VALIDATION
    
#     model.train()
    start = time.time()
    temp = start
#     total_loss = 0
    
    for epoch in range(epochs):
        
        model.train()      
#         start = time.time()
#         temp = start
        total_loss_train = 0
        
        for i, batch in enumerate(train_iter):            
            src = batch.Complicated.transpose(0,1)
            trg = batch.Simple.transpose(0,1)            
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            src_mask, trg_mask = create_masks(src, trg_input)
            
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), targets)            
            loss.backward()
            optim.step()
            
            # LOSS.DATA WAS INDEXED [0] BUT BECAUSE ONLY 1 LINE TAKEN, INDEX REMOVED.
            # MAY LEAD TO INCONSISTENCY. TAKE CARE
            total_loss_train += loss.data
            if (i + 1) % print_every == 0:
                loss_avg_train = total_loss_train / print_every
                print("TRAINING LOSS: time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60, epoch + 1, i + 1, loss_avg_train, time.time() - temp, print_every))
                total_loss_train = 0
                temp = time.time()
                
        # VALIDATION
        model.eval()
#         start = time.time()
#         temp = start
        total_loss_val = 0
  
        for i, batch in enumerate(val_iter):            
            src = batch.Complicated.transpose(0,1)
            trg = batch.Simple.transpose(0,1)            
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            src_mask, trg_mask = create_masks(src, trg_input)
            
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), targets)            
            loss.backward()
            optim.step()
            
            # LOSS.DATA WAS INDEXED [0] BUT BECAUSE ONLY 1 LINE TAKEN, INDEX REMOVED.
            # MAY LEAD TO INCONSISTENCY. TAKE CARE
            total_loss_val += loss.data
            if (i + 1) % print_every == 0:
                loss_avg_val = total_loss_val / print_every
                print("VALIDATION LOSS: time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60, epoch + 1, i + 1, loss_avg_val, time.time() - temp, print_every))
                total_loss_val = 0
                temp = time.time()
    

## TRAINING AND VALIDATING MODEL

In [0]:
train_model(1)

TRAINING LOSS: time = 0m, epoch 1, iter = 1, loss = 11.950, 17s per 1 iters


## INFERENCE

In [0]:
def simplify(model, src, max_len = 100, custom_string=True):
    
    sentence = ""
    model.eval()
    
    
    # REQUIRES FILE PATH INPUT
    # THEN TOKENISATION AND BATCH CREATION ALSO MAYBE. LOOKEY HERE:
    #             src = batch.Complicated.transpose(0,1)
    #             trg = batch.Simple.transpose(0,1)    
    
    
    if custom_string == True:
        src = tokenize_complicated(src)
        sentence=\
        Variable(torch.LongTensor([[C_TEXT.vocab.stoi[tok] for tok
        in sentence]])).cuda()
#         print(src)
        
    src_mask = (src != input_pad).unsqueeze(-2)
    e_outputs = model.encoder(src, src_mask)
    
    outputs = torch.zeros(max_len).type_as(src.data)
    outputs[0] = torch.LongTensor([S_TEXT.vocab.stoi['<sos>']])
    
    for i in range(1, max_len):    
            
        # TAKE CARE HERE FOR NP.ONES PARAMETER LIST CHANGED.
        # ORIGINALLY WAS TRG_MASK BOTH OF THEM. TRY INTERCHANGING IF PROBLEMS ARISE
        src_mask = np.triu(np.ones(shape = (1, i, i),k=1)).astype('uint8')
        trg_mask = Variable(torch.from_numpy(trg_mask) == 0).cuda()
        
        out = model.out(model.decoder(outputs[:i].unsqueeze(0), e_outputs, src_mask, trg_mask))
        out = F.softmax(out, dim=-1)
        val, ix = out[:, -1].data.topk(1)
        
        outputs[i] = ix[0][0]
        if ix[0][0] == S_TEXT.vocab.stoi['<eos>']:
            break
                           
    return ' '.join(
    [S_TEXT.vocab.itos[ix] for ix in outputs[:i]])

## GIVING INPUT TO INFERENCE

In [0]:
simplify(model, path + "test.en")

In [0]:
import sys
print(sys.maxsize)